In [1]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
gdf = gpd.read_file('./Maps/mapa_energetico_abc.json')

gdf['ANNUAL'] = pd.to_numeric(gdf['ANNUAL'], errors='coerce')
gdf = gdf.dropna(subset=['ANNUAL'])

In [3]:
fsa_lat = -23.661280
fsa_lon = -46.554371

# Criar o mapa base centrado no ponto de interesse
m = folium.Map(location=[fsa_lat, fsa_lon], zoom_start=12, tiles='CartoDB positron')
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)

# Criar um colormap usando matplotlib
cmap = plt.get_cmap('OrRd')
norm = colors.Normalize(vmin=gdf['ANNUAL'].min(), vmax=gdf['ANNUAL'].max())

def style_function(feature):
    value = feature['properties']['ANNUAL']
    color = colors.to_hex(cmap(norm(value)))
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    }

# Adicionar o GeoDataFrame ao mapa
folium.GeoJson(
    gdf,
    name='Irradiação solar',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['ANNUAL'],
        aliases=['Média kWh/m².dia'],
        localize=True
    )
).add_to(m)

# Adicionar o ponto de interesse (FSA) ao mapa
folium.Marker(
    location=[fsa_lat, fsa_lon],
    popup="FSA",
    icon=folium.Icon(color='blue')
).add_to(m)

# Adicionar camada de controle
folium.LayerControl().add_to(m)

# Salvar o mapa como um arquivo HTML
m.save('mapa_interativo.html')